In [65]:
import pandas as pd
import numpy as np
import json
from flatten_json import flatten
import time
import atexit
from datetime import datetime
from datetime import date
from tda.client import Client
from tda.auth import easy_client
from tda import auth
from tda.orders import equities
from tda.orders.common import Duration, Session

In [156]:
# Set account variables and webdriver path
account_id = open('/mnt/c/Python/API Keys/TD/TD_ACCOUNT_ID.txt').read()
rothira_account_id = open('/mnt/c/Python/API Keys/TD/TD_ROTHIRA_ACCOUNT_ID.txt').read()
traditionalira_account_id = open('/mnt/c/Python/API Keys/TD/TD_TRADITIONALIRA_ACCOUNT_ID.txt').read()
consumer_key = open('/mnt/c/Python/API Keys/TD/TD_CONSUMER_KEY.txt').read()
redirect_uri = 'http://localhost'
token_path = '/mnt/c/Python/API Keys/TD/ameritrade-credentials.json'
geckodriver_path = '/usr/local/bin/geckodriver'
# Creates Webdriver for Selenium
def make_webdriver():
    # Import selenium here because it's slow to import
    from selenium import webdriver
    driver = webdriver.Firefox(executable_path = geckodriver_path)
    atexit.register(lambda: driver.quit())
    return driver

In [3]:
# Create client object
c = easy_client(consumer_key,
                            redirect_uri,
                            token_path,
                            make_webdriver)

In [194]:
class TDformatter:
    
    class Quote_formatter:
        """
        Returns formatted json of quotes.
        """
        def __init__(self, response):
            self.data = response.json()

        def to_df(self):
            """
            Returns pandas DataFrame from formatted json of
            quotes created with Quote_formatter.
            """
            return pd.DataFrame(self.data).T
    
    
    class History_formatter:
        """
        Returns formatted json of history. 
        """
        def __init__(self, response):
            self.data = response.json()
            self.candles = self.data['candles']
            self.ticker = self.data['symbol']

        def to_df(self):
            """
            Returns pandas DataFrame from formatted json of
            history created with History_formatter.
            """
            self.dataframe = pd.DataFrame(self.candles)
            self.dataframe['symbol'] = self.ticker
            self.dataframe.set_index('datetime', inplace=True)
#             Converts time since epoch to a datetime object with second accuracy (any more is unneccessary)
#             and sets it as index.
            self.dataframe.index = pd.to_datetime((self.dataframe.index/1000).astype('int64'), unit='s')
            return self.dataframe
        
        
    class Chain_formatter:
        """
        Returns formatted json of options chains.  Can take call &/or put chains, if given 
        both will return tuple of calls, puts.
        """
        def __init__(self, response):
            self.data = response.json()
#             Check if the options_chain response contains calls, if empty then set flag to false
            if self.data['callExpDateMap'] != {}:
                self.calls = self.data['callExpDateMap'].values()
                self.calls_exists = True
            else:
                self.calls_exists = False
            
#             Check if the options_chain response contains puts, if empty then set flag to false.
            if self.data['putExpDateMap'] != {}:
                self.puts = self.data['putExpDateMap'].values()
                self.puts_exists = True
            else:
                self.puts_exists = False
        
        def to_df(self):
            """
            Returns dataframe from formatted json of
            options chain created with Chain_formatter.
            """
#             Gets today's date for use in index.
            today = datetime.now().strftime('%m-%d-%Y')
            
#             Checks if options_chain response contains calls, if not then skips.
            if self.calls_exists:
                self.calls_list = []
#             **Very inefficient triple for loop, should be revised if possible**
                for i in self.calls:
                    for j in i.values():
                        for k in j:
                            self.calls_list.append(k)
                self.calls_df = pd.DataFrame(self.calls_list)
                self.calls_df.set_index('description', inplace=True)
                self.calls_df['Date'] = today
#                 Removes garbage responses with -999 deltas
                self.calls_df[self.calls_df['delta'] != -999.0]
            
#             Checks if options_chain response contains puts, if not then skips.
            if self.puts_exists:
                self.puts_list = []
#             **Very inefficient triple for loop, should be revised if possible**
                for i in self.puts:
                    for j in i.values():
                        for k in j:
                            self.puts_list.append(k)
                self.puts_df = pd.DataFrame(self.puts_list)
                self.puts_df.set_index('description', inplace=True)
                self.puts_df['Date'] = today
#                 Removes garbage responses with -999 deltas
                self.puts_df[self.puts_df['delta'] != -999.0]
            
#            Determines return values based on if calls &/or puts exists.
            if self.calls_exists and self.puts_exists:
                return self.calls_df, self.puts_df
            elif self.calls_exists and not self.puts_exists:
                return self.calls_df
            elif not self.calls_exists and self.puts_exists:
                return self.puts_df


    class Account_formatter:
        """
        Returns formatted json of account info.
        """
        def __init__(self, response, accounts_namemap=None):
            self.data = response.json()
            self.accounts_namemap = accounts_namemap
        
        def to_df(self):
            """
            Returns pandas DataFrame from formatted json of
            account_info created with Account_formatter.
            """
            account_df_list = []
            for d in self.data:
                account_info_json = flatten(d['securitiesAccount'], '.')
                df = pd.DataFrame(account_info_json, index=[0])
                if self.accounts_namemap:
                    for pair in self.accounts_namemap.items():
                        if df['accountId'][0] == pair[1]:
                            df['accountId'] = pair[0]
                df.set_index('accountId', inplace=True)
                account_df_list.append(df)
            account_info_df = pd.concat(account_df_list)
            new_labels = [name.replace('.0', '') for name in list(account_info_df.columns)]
            account_info_df.columns = new_labels
            return account_info_df

In [5]:
symbol_list = ['SPY', 'QQQ']

In [6]:
quote_response = c.get_quotes(symbol_list)
TDformatter().Quote_formatter(quote_response).to_df()

,assetType,assetMainType,cusip,assetSubType,symbol,description,bidPrice,bidSize,bidId,askPrice,...,regularMarketLastPrice,regularMarketLastSize,regularMarketNetChange,regularMarketTradeTimeInLong,netPercentChangeInDouble,markChangeInDouble,markPercentChangeInDouble,regularMarketPercentChangeInDouble,delayed,realtimeEntitled
SPY,ETF,EQUITY,78462F103,ETF,SPY,SPDR S&P 500,452.71,100,P,452.76,...,453.08,0,0.0,1630713600065,0.0,0.0,0.0,0.0,False,True
QQQ,ETF,EQUITY,46090E103,ETF,QQQ,"Invesco QQQ Trust, Series 1",381.42,200,P,381.45,...,381.57,1939,0.0,1630699200802,-0.0314,0.0,0.0,0.0,False,True


In [7]:
history_dict = {}
for symbol in symbol_list:
    history_response = c.get_price_history(symbol = symbol,
                                        period_type = Client.PriceHistory.PeriodType.YEAR,
                                        period = Client.PriceHistory.Period.TWENTY_YEARS,
                                        frequency_type = Client.PriceHistory.FrequencyType.DAILY,
                                        frequency = Client.PriceHistory.Frequency.DAILY)
    history_dict[symbol] = TDformatter.History_formatter(history_response).to_df()

In [8]:
chain_dict = {}
for symbol in symbol_list:
    chain_response = c.get_option_chain(symbol = symbol,
                      contract_type = Client.Options.ContractType.CALL,
                      strike_range = Client.Options.StrikeRange.ALL)
    chain_dict[symbol] = TDformatter().Chain_formatter(chain_response).to_df()

In [197]:
account_info = c.get_accounts(fields = [c.Account.Fields.POSITIONS, c.Account.Fields.ORDERS])
accounts_namemap = {'Main' : account_id, 'RothIRA': rothira_account_id, 'TraditionalIRA': traditionalira_account_id}
TDformatter.Account_formatter(account_info, accounts_namemap).to_df()

,type,roundTrips,isDayTrader,isClosingOnlyRestricted,positions.shortQuantity,positions.averagePrice,positions.currentDayCost,positions.currentDayProfitLoss,positions.currentDayProfitLossPercentage,positions.longQuantity,...,currentBalances.bondValue,projectedBalances.availableFunds,projectedBalances.availableFundsNonMarginableTrade,projectedBalances.buyingPower,projectedBalances.dayTradingBuyingPower,projectedBalances.dayTradingBuyingPowerCall,projectedBalances.maintenanceCall,projectedBalances.regTCall,projectedBalances.isInCall,projectedBalances.stockBuyingPower
accountId,,,,,,,,,,,,,,,,,,,,,
TraditionalIRA,MARGIN,0,False,False,0.0,0.0,0.0,0.0,0.0,50.00,...,0.0,50.00,50.00,50.00,0.0,0.0,0.0,0.0,False,50.00
RothIRA,MARGIN,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,False,0.00
Main,MARGIN,0,False,False,0.0,1.0,0.0,0.0,0.0,36.59,...,0.0,36.59,36.59,36.59,0.0,0.0,0.0,0.0,False,36.59
